In [1]:
# In order to load environment variables, import these 2 libraries:
import os
from dotenv import load_dotenv

In [2]:
# Load your environment variables
load_dotenv('_.env')  # Load environment variables from the file '.env'
API_KEY_FMP = os.environ.get('API_KEY_FMP')  # Retrieve the value of the environment variable 'API_KEY_FMP'   
API_KEY_FRED = os.environ.get('API_KEY_FRED')  # Retrieve the value of the environment variable 'API_KEY_FRED'

In [3]:
# Import the requests library
import requests

# Construct your API request endpoint url
BASE_URL = 'https://financialmodelingprep.com/api/v3'
company_tick = "AAPL"
endpoint_url_dividend = f"{BASE_URL}/historical-price-full/stock_dividend/{company_tick}?apikey={API_KEY_FMP}"

endpoint_url_ratios = f"{BASE_URL}/key-metrics/{company_tick}?apikey={API_KEY_FMP}"

In [4]:
# Send an HTTP GET request to the endpoint URL and store the response
response = requests.get(endpoint_url_dividend)
if response.status_code == 429:
    print("FMP API limit reached")
print(type(response))

<class 'requests.models.Response'>


In [5]:
# Convert json to dictionary object and then a Pandas Dataframe
import pandas as pd
response_dict = response.json()
dividends = pd.DataFrame(response_dict['historical'])

In [6]:
print(dividends) 

          date            label  adjDividend  dividend  recordDate  \
0   2024-05-10       May 10, 24     0.250000      0.25  2024-05-13   
1   2024-02-09  February 09, 24     0.240000      0.24  2024-02-12   
2   2023-11-10  November 10, 23     0.240000      0.24  2023-11-13   
3   2023-08-11    August 11, 23     0.240000      0.24  2023-08-14   
4   2023-05-12       May 12, 23     0.240000      0.24  2023-05-15   
..         ...              ...          ...       ...         ...   
78  1988-05-16       May 16, 88     0.000714      0.08  1988-05-20   
79  1988-02-12  February 12, 88     0.000714      0.08  1988-02-19   
80  1987-11-17  November 17, 87     0.000714      0.08  1987-11-23   
81  1987-08-10    August 10, 87     0.000536      0.06  1987-08-14   
82  1987-05-11       May 11, 87     0.000536      0.12  1987-05-15   

   paymentDate declarationDate  
0   2024-05-16      2024-05-02  
1   2024-02-15      2024-02-01  
2   2023-11-16      2023-11-02  
3   2023-08-17      2023-08

In [7]:
print(response_dict)

{'symbol': 'AAPL', 'historical': [{'date': '2024-05-10', 'label': 'May 10, 24', 'adjDividend': 0.25, 'dividend': 0.25, 'recordDate': '2024-05-13', 'paymentDate': '2024-05-16', 'declarationDate': '2024-05-02'}, {'date': '2024-02-09', 'label': 'February 09, 24', 'adjDividend': 0.24, 'dividend': 0.24, 'recordDate': '2024-02-12', 'paymentDate': '2024-02-15', 'declarationDate': '2024-02-01'}, {'date': '2023-11-10', 'label': 'November 10, 23', 'adjDividend': 0.24, 'dividend': 0.24, 'recordDate': '2023-11-13', 'paymentDate': '2023-11-16', 'declarationDate': '2023-11-02'}, {'date': '2023-08-11', 'label': 'August 11, 23', 'adjDividend': 0.24, 'dividend': 0.24, 'recordDate': '2023-08-14', 'paymentDate': '2023-08-17', 'declarationDate': '2023-08-03'}, {'date': '2023-05-12', 'label': 'May 12, 23', 'adjDividend': 0.24, 'dividend': 0.24, 'recordDate': '2023-05-15', 'paymentDate': '2023-05-18', 'declarationDate': '2023-05-04'}, {'date': '2023-02-10', 'label': 'February 10, 23', 'adjDividend': 0.23, '

In [8]:
#design preferred date range  

start_year = 2013  
end_year = 2022  

end_year = end_year + 1  

list(range(start_year, end_year))

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [9]:
# Data Transformation
if dividends.shape == (0, 0):  # Handle the case where the company never issued any dividend in the past
    dividends = pd.DataFrame({
        "year": list(range(start_year - 1, end_year + 1)),
        "adjDividend": [0.0] * len(list(range(start_year - 1, end_year + 1)))  # We are obtaining 2 more years' data
    })
else:
    # Extract year data from the date column
    dividends['year'] = pd.to_datetime(dividends['date']).dt.year
    # Aggregate the dividend paid by year
    dividends = dividends.groupby("year").agg({"adjDividend": "sum"}).reset_index()
    # Create a new DataFrame with all years from start to end - So that we don't omit years without dividends
    all_years = pd.DataFrame({'year': list(range(start_year - 1, end_year + 1))})
    # Merge the two DataFrames on the year column and fill missing values with 0.0
    dividends = all_years.merge(dividends, on='year', how='left').fillna(0.0)

In [10]:
# Let's create our target variable
import numpy as np

dividends['next_year_dividend'] = dividends['adjDividend'].shift(-1)

conditions = [
    dividends['adjDividend'] <= dividends['next_year_dividend'],
    dividends['adjDividend'] > dividends['next_year_dividend']
]

choices = ['constant/increased', 'decreased']      

# Create the target column 'dps_change' based on the conditions
dividends['dps_change_next_year'] = np.select(conditions, choices, default=np.nan)

In [11]:
# We can also create a predictor here - dps change from last year
dividends['last_year_dividend'] = dividends['adjDividend'].shift(1)
dividends['dps_growth'] = dividends['adjDividend'] - dividends['last_year_dividend']

In [12]:
print(dividends)

    year  adjDividend  next_year_dividend dps_change_next_year  \
0   2012     0.189286            0.421433   constant/increased   
1   2013     0.421433            0.461430   constant/increased   
2   2014     0.461430            0.507500   constant/increased   
3   2015     0.507500            0.557500   constant/increased   
4   2016     0.557500            0.615000   constant/increased   
5   2017     0.615000            0.705000   constant/increased   
6   2018     0.705000            0.760000   constant/increased   
7   2019     0.760000            0.807500   constant/increased   
8   2020     0.807500            0.865000   constant/increased   
9   2021     0.865000            0.910000   constant/increased   
10  2022     0.910000            0.950000   constant/increased   
11  2023     0.950000                 NaN                  nan   

    last_year_dividend  dps_growth  
0                  NaN         NaN  
1             0.189286    0.232147  
2             0.421433    0.03

In [13]:
# Another predictor that we can create is dividend change as a percentage
dividends['dps_growth_rate'] = np.where(
    (dividends['last_year_dividend'] == 0) & (dividends['adjDividend'] == 0),
    0,  # If both are 0 then change is 0
    np.where(
        dividends['last_year_dividend'] != 0,
        ((dividends['adjDividend'] / dividends['last_year_dividend']) - 1) * 100,
        999  # If last year dividend is 0 then return 999
    )
)

In [14]:
# Remove the first last year since they will be NaN
dividends = dividends.loc[(dividends['year'] >= start_year) & (dividends['year'] <= end_year - 1)]
# Only keep the columns that we need
dividends = dividends[["year", "adjDividend", "dps_growth", "dps_growth_rate", "dps_change_next_year"]]

In [15]:
print(dividends)

    year  adjDividend  dps_growth  dps_growth_rate dps_change_next_year
1   2013     0.421433    0.232147       122.643951   constant/increased
2   2014     0.461430    0.039997         9.490776   constant/increased
3   2015     0.507500    0.046070         9.984180   constant/increased
4   2016     0.557500    0.050000         9.852217   constant/increased
5   2017     0.615000    0.057500        10.313901   constant/increased
6   2018     0.705000    0.090000        14.634146   constant/increased
7   2019     0.760000    0.055000         7.801418   constant/increased
8   2020     0.807500    0.047500         6.250000   constant/increased
9   2021     0.865000    0.057500         7.120743   constant/increased
10  2022     0.910000    0.045000         5.202312   constant/increased


In [16]:
# Engineer some other predictors
predictors = pd.DataFrame({"year": list(range(start_year - 1, end_year))})  # We include one more year before
                                                                            # the first year to calculate changes

In [17]:
print(predictors)

    year
0   2012
1   2013
2   2014
3   2015
4   2016
5   2017
6   2018
7   2019
8   2020
9   2021
10  2022


In [18]:
# Let's include the Company's Industry and sector data
import yfinance as yf
company_data_raw = yf.Ticker(company_tick)
company_data = company_data_raw.info
predictors["industry"] = company_data['industry']
predictors["sector"] = company_data['sector']

In [19]:
print(predictors)

    year              industry      sector
0   2012  Consumer Electronics  Technology
1   2013  Consumer Electronics  Technology
2   2014  Consumer Electronics  Technology
3   2015  Consumer Electronics  Technology
4   2016  Consumer Electronics  Technology
5   2017  Consumer Electronics  Technology
6   2018  Consumer Electronics  Technology
7   2019  Consumer Electronics  Technology
8   2020  Consumer Electronics  Technology
9   2021  Consumer Electronics  Technology
10  2022  Consumer Electronics  Technology


In [20]:
# Let's add more Predictors -- dividend payout ratio, return on equity, board composition(gender and title) and dividend yield

# Construct the API request endpoint url
BASE_URL = 'https://financialmodelingprep.com/api/v3'
company_tick = "AAPL"

endpoint_ratios = f"{BASE_URL}/ratios/{company_tick}?apikey={API_KEY_FMP}"    
endpoint_ceo = f"{BASE_URL}/key-executives/{company_tick}?apikey={API_KEY_FMP}"    
endpoint_yield  = f"{BASE_URL}/key-metrics/{company_tick}?apikey={API_KEY_FMP}"  

In [21]:
# Send an HTTP GET request to the endpoint URL and store the response
end_response = requests.get(endpoint_ratios)
if end_response.status_code == 429:
    print("FMP API limit reached")
print(type(end_response))  
  
end_response1 = requests.get(endpoint_ceo)
if end_response1.status_code == 429:
    print("FMP API limit reached")
print(type(end_response1))    

end_response2 = requests.get(endpoint_yield)  
if end_response1.status_code == 429:
    print("FMP API limit reached")
print(type(end_response2))      

end_response3 = requests.get(endpoint_yield)  
if end_response1.status_code == 429:
    print("FMP API limit reached")
print(type(end_response2))     

<class 'requests.models.Response'>
<class 'requests.models.Response'>
<class 'requests.models.Response'>
<class 'requests.models.Response'>


In [22]:
# Convert json to dictionary object and then a Pandas Dataframe
import pandas as pd
response_dicts = end_response.json()  
payout =  pd.DataFrame(response_dicts)  

response_dicts2 = end_response1.json()  
board_compose = pd.DataFrame(response_dicts2)    

response_dicts3 = end_response2.json()  
div_yield = pd.DataFrame(response_dicts3)      

response_dicts4 = end_response3.json()  
roe = pd.DataFrame(response_dicts4)      

In [23]:
print(payout)

   symbol        date calendarYear period  currentRatio  quickRatio  \
0    AAPL  2023-09-30         2023     FY      0.988012    0.843312   
1    AAPL  2022-09-24         2022     FY      0.879356    0.709408   
2    AAPL  2021-09-25         2021     FY      1.074553    0.909660   
3    AAPL  2020-09-26         2020     FY      1.363604    1.218195   
4    AAPL  2019-09-28         2019     FY      1.540126    1.384447   
5    AAPL  2018-09-29         2018     FY      1.123843    0.986566   
6    AAPL  2017-09-30         2017     FY      1.276063    1.089670   
7    AAPL  2016-09-30         2016     FY      1.352669    1.220844   
8    AAPL  2015-09-30         2015     FY      1.108771    0.892495   
9    AAPL  2014-09-30         2014     FY      1.080113    0.824234   
10   AAPL  2013-09-30         2013     FY      1.678639    1.401507   
11   AAPL  2012-09-30         2012     FY      1.495849    1.240750   
12   AAPL  2011-09-30         2011     FY      1.608438    1.346764   
13   A

In [24]:
print(board_compose)

                                               title                     name  \
0  Senior Vice President, General Counsel & Secre...   Ms. Katherine L. Adams   
1                      Head of Corporate Development       Mr. Adrian  Perica   
2                     Director of Investor Relations   Suhasini  Chandramouli   
3       Senior Vice President of Worldwide Marketing        Mr. Greg  Joswiak   
4            Senior Director of Corporate Accounting         Mr. Chris  Kondo   
5                 Chief Executive Officer & Director      Mr. Timothy D. Cook   
6    Chief Financial Officer & Senior Vice President        Mr. Luca  Maestri   
7                            Chief Operating Officer  Mr. Jeffrey E. Williams   
8                    Senior Vice President of Retail     Ms. Deirdre  O'Brien   
9                           Chief Technology Officer        Mr. James  Wilson   

          pay currencyPay  gender  yearBorn titleSince  
0   4618064.0         USD  female    1964.0       N

In [25]:
print(div_yield)

   symbol        date calendarYear period  revenuePerShare  netIncomePerShare  \
0    AAPL  2023-09-30         2023     FY        24.344473           6.160669   
1    AAPL  2022-09-24         2022     FY        24.317273           6.154614   
2    AAPL  2021-09-25         2021     FY        21.903541           5.669029   
3    AAPL  2020-09-26         2020     FY        15.820258           3.308587   
4    AAPL  2019-09-28         2019     FY        14.085283           2.991446   
5    AAPL  2018-09-29         2018     FY        13.399334           3.003354   
6    AAPL  2017-09-30         2017     FY        10.984444           2.316885   
7    AAPL  2016-09-30         2016     FY         9.854053           2.087758   
8    AAPL  2015-09-30         2015     FY        10.155480           2.320098   
9    AAPL  2014-09-30         2014     FY         7.509360           1.623101   
10   AAPL  2013-09-30         2013     FY         6.596481           1.429488   
11   AAPL  2012-09-30       

In [26]:
print(roe)

   symbol        date calendarYear period  revenuePerShare  netIncomePerShare  \
0    AAPL  2023-09-30         2023     FY        24.344473           6.160669   
1    AAPL  2022-09-24         2022     FY        24.317273           6.154614   
2    AAPL  2021-09-25         2021     FY        21.903541           5.669029   
3    AAPL  2020-09-26         2020     FY        15.820258           3.308587   
4    AAPL  2019-09-28         2019     FY        14.085283           2.991446   
5    AAPL  2018-09-29         2018     FY        13.399334           3.003354   
6    AAPL  2017-09-30         2017     FY        10.984444           2.316885   
7    AAPL  2016-09-30         2016     FY         9.854053           2.087758   
8    AAPL  2015-09-30         2015     FY        10.155480           2.320098   
9    AAPL  2014-09-30         2014     FY         7.509360           1.623101   
10   AAPL  2013-09-30         2013     FY         6.596481           1.429488   
11   AAPL  2012-09-30       

In [27]:
predictors['dividendPayoutRatio'] = payout['dividendPayoutRatio']  
predictors['boardtitle'] = board_compose['title']  
predictors['boardgender'] = board_compose['gender']    
predictors['dividendYield'] = div_yield['dividendYield']  
predictors['roe'] = div_yield['roe']

In [28]:
print(predictors)

    year              industry      sector  dividendPayoutRatio  \
0   2012  Consumer Electronics  Technology             0.154905   
1   2013  Consumer Electronics  Technology             0.148703   
2   2014  Consumer Electronics  Technology             0.152799   
3   2015  Consumer Electronics  Technology             0.245267   
4   2016  Consumer Electronics  Technology             0.255520   
5   2017  Consumer Electronics  Technology             0.230334   
6   2018  Consumer Electronics  Technology             0.264090   
7   2019  Consumer Electronics  Technology             0.265940   
8   2020  Consumer Electronics  Technology             0.216522   
9   2021  Consumer Electronics  Technology             0.281600   
10  2022  Consumer Electronics  Technology             0.285228   

                                           boardtitle boardgender  \
0   Senior Vice President, General Counsel & Secre...      female   
1                       Head of Corporate Development    

In [29]:
def calculate_change(df, feature_name):
    percentage_change = df[feature_name].pct_change() * 100
    # Create new column name
    new_col_name = f"{feature_name}_percentage_change"
    # Find the index position of the original predictor column
    original_col_position = df.columns.get_loc(feature_name)
    # Insert the new column right after the original predictor column
    df.insert(original_col_position + 1, new_col_name, percentage_change)

In [30]:
feature_list = list(predictors.columns)
feature_list.remove('year')
feature_list.remove('industry')
feature_list.remove('sector')  
feature_list.remove('boardtitle')  
feature_list.remove('boardgender')

In [31]:
for feature in feature_list:
    calculate_change(predictors, feature)

# Replacing inf and NaN values
predictors.replace([float('inf'), float('-inf')], 999, inplace=True)
predictors.fillna(0, inplace=True)

# Combine dividend data with other predictors
dataset = pd.merge(dividends, predictors, left_on='year', right_on='year', how='left')

# Move target to the end of the dataset for good practice
feature_list = list(dataset.columns)
feature_list.append('dps_change_next_year')
feature_list.remove('dps_change_next_year')
dataset = dataset[feature_list]

# Return the modified dataset
dataset

,year,adjDividend,dps_growth,dps_growth_rate,industry,sector,dividendPayoutRatio,dividendPayoutRatio_percentage_change,boardtitle,boardgender,dividendYield,dividendYield_percentage_change,roe,roe_percentage_change,dps_change_next_year
0,2013,0.421433,0.232147,122.643951,Consumer Electronics,Technology,0.148703,-4.003713,Head of Corporate Development,male,0.006084,9.149579,1.969589,26.194197,constant/increased
1,2014,0.461430,0.039997,9.490776,Consumer Electronics,Technology,0.152799,2.754456,Director of Investor Relations,,0.005896,-3.091461,1.500713,-23.805755,constant/increased
2,2015,0.507500,0.046070,9.984180,Consumer Electronics,Technology,0.245267,60.515936,Senior Vice President of Worldwide Marketing,male,0.007227,22.583090,0.878664,-41.450269,constant/increased
3,2016,0.557500,0.050000,9.852217,Consumer Electronics,Technology,0.255520,4.180421,Senior Director of Corporate Accounting,male,0.013971,93.312678,0.610645,-30.503037,constant/increased
4,2017,0.615000,0.057500,10.313901,Consumer Electronics,Technology,0.230334,-9.856767,Chief Executive Officer & Director,male,0.012257,-12.272007,0.555601,-9.013972,constant/increased
5,2018,0.705000,0.090000,14.634146,Consumer Electronics,Technology,0.264090,14.655211,Chief Financial Officer & Senior Vice President,male,0.015880,29.562643,0.360702,-35.079000,constant/increased
6,2019,0.760000,0.055000,7.801418,Consumer Electronics,Technology,0.265940,0.700635,Chief Operating Officer,male,0.019647,23.718543,0.356237,-1.237905,constant/increased
7,2020,0.807500,0.047500,6.250000,Consumer Electronics,Technology,0.216522,-18.582210,Senior Vice President of Retail,female,0.018214,-7.290702,0.447355,25.577891,constant/increased
8,2021,0.865000,0.057500,7.120743,Consumer Electronics,Technology,0.281600,30.055607,Chief Technology Officer,male,0.018145,-0.382762,0.354200,-20.823319,constant/increased
9,2022,0.910000,0.045000,5.202312,Consumer Electronics,Technology,0.285228,1.288599,0,0,0.023942,31.949913,0.299776,-15.365500,constant/increased
